# Question 4: General Theory/Application

_No need to be verbose, it's not fun for anyone_

### 1. What part of S**L**A**M** did this project deal with? Why? What does the other part deal with and how would it generally work, given that you only have LIDAR scans, RGB video stream, and noisy pose data for a moving robot?

We dealt with localisation in this project. We dealt with the estimate of the robot's pose with respect to some map and dealing with the noise in those poses.
The other part is mapping. It deals with the collection and connecting different sensor measurements into a common map with the help of LIDAR scans, RGB video Stream, noisy pose data.


### 2. Loop closures play an important role in reducing drift, how would you go about detecting these?

Loop closure can be detected using **visual place recognition**. In this technique entire database of images is parsed to find the best match for the current image. The essential condition for this algorithm is that the current image is actually a revisited place and a reference image is present in our database.

<img src="../data/loop_closure.png" alt="Drawing"/>

Let's consider a case of a robot moving in an indoor space, as shown in the image. Here when we look at pose number 8, we have more constraints than usual. Now for each pose least square error of the global graph is minimized, to reduce further the effect of outliers, robust cost functions (like Huber loss) are used.

Once we are sure about the loop closure, it is likely that our current image/pose remains unadjusted and rather adjust the poses of all non-active frame poses.

### 3. Explain the structure of your Jacobian. Is the pose-graph fully connected? Why/Why not?

The jacobian of residual(k,j) w.r.t. a pose k is ,
                            
$$ \mathbf{J_{s(k,j)}} =  \left[\begin{array}{ccc}
1 & 0 & -\Delta x_{(k,j)} \sin(\theta_k) - \Delta y_{(k,j)} \cos(\theta_k) \\
0 & 1 & -\Delta y_{(k,j)} \sin(\theta_k) + \Delta x_{(k,j)} \cos(\theta_k) \\
0 & 0 & 1 \\
\end{array}\right]$$
</br>
Also the jacobian of the same residual w.r.t. pose j is,
$$ \mathbf{J_{n(k,j)}} =  \left[\begin{array}{ccc}
-1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & -1 \\
\end{array}\right]$$
</br>
The Jocabian is given by,
<img src="../data/jacobian_structure.png" alt="Drawing"/>
The Shape of the jacobian is (m+n+1) * N x (m+1) * N,</br>
where m is no. of poses(vertices), 1 prior , hence (m+1) poses</br>
n is no. of loop closure constraints, N is no. of variables. 

Shape in our case is 420 by 360.

The Jacobian matrix is sparse in nature, only filled for 2 * N entries per row (except last N rows, which represent prior). We can also observe that the columns filled represent the edges between poses (both normal and loop closure).

The shape is kind off diagonal for the part of matrix which represents adjacent edges.
</br>
</br>
</br>
**Is the pose-graph fully connected? Why/Why not?**

No, the graph is not fully connected. We get a sparse one as the sensor range is limited to the robot specifications.

<img src="../data/sparse.png" alt="Drawing"/>

Source: Factor Graphs for Robot Perception, Daellart and Kaess, Figure 2.1

### 4. With what you know now, how would you describe and differentiate the SLAM frontend and backend? Why do we need to optimise our poses/map in the first place - where does the noise come from/why?
Front-end is to get the raw data from the sensors (laser-based, RGBD, etc.) and convert that to a graph. It is used mainly for **GRAPH CONSTRUCTION**. 
The main task includes -
1. Odometry information:- New vertex in a graph is added according to the odometry constraints
2. Handling loop closures:- Analysing the whole trajectory, the front end should detect any place which has been visited multiple times and generate constraints accordingly.
3. Removing outliers

The back end is used to optimize the initial graph proposed by the front end using odometry information. It is used primarily for **GRAPH OPTIMISATION**. The main task is to 
1. Find the Maximum A-posteriori estimate that explains all the observation
2. Reduce the least square error

If we get perfect odometry information, then there is no need of SLAM for optimization. But we know there is always some noise present in the odometry information, These algorithms work well in a local environment, but on a global scale, these drift in odometry will lead to significant errors; therefore optimization of poses/maps using SLAM is required.

Generally, this odometry data is extracted from the visual information, i.e. difference of the current image to the previous image, which gives us the incremental change in the pose, which ultimately leads to the drift after several readings.